[View in Colaboratory](https://colab.research.google.com/github/sprotou/samplePythonJupyter1/blob/master/Spyros_Prot_DecMakingHW1.ipynb)

In [0]:
#@title Exercise1


In [5]:
!pip install cvxpy
!pip install cvxopt

    100% |████████████████████████████████| 163kB 7.7MB/s 
    100% |████████████████████████████████| 122kB 7.2MB/s 
    100% |████████████████████████████████| 143kB 7.9MB/s 
    100% |████████████████████████████████| 1.5MB 11.4MB/s 
    100% |████████████████████████████████| 51kB 17.5MB/s 
    100% |████████████████████████████████| 696kB 14.3MB/s 
    100% |████████████████████████████████| 71kB 20.4MB/s 
  Running setup.py bdist_wheel for cvxpy ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/2d/26/ba/f0a0388a7ab98124caf88ea8b532b81cf4acb1b6a4a990e3b4
  Running setup.py bdist_wheel for ecos ... - \ | / - \ | / - \ | / - \ | / - \ | /

 - \ | / done
  Stored in directory: /content/.cache/pip/wheels/50/91/1b/568de3c087b3399b03d130e71b1fd048ec072c45f72b6b6e9a
  Running setup.py bdist_wheel for scs ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/ff/f0/aa/530ccd478d7d9900b4e9ef5bc5a39e895ce110bed3d3ac653e
  Running setup.py bdist_wheel for multiprocess ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/85/44/92/7635fa709a1e88a4dca84eba3d1566fc0e2b001d4d84f40d35
  Running setup.py bdist_wheel for fastcache ... - \ done
  Stored in directory: /content/.cache/pip/wheels/b7/90/c0/da92ac52d188d9ebca577044e89a14d0e6ff333c1bcd1ebc14
  Running setup.py bdist_wheel for toolz ... - \ done
  Stored in directory: /content/.cache/pip/wheels/f4/0c/f6/ce6b2d1aa459ee97cc3c0f82236302bd62d89c86c700219463
  Running setup.py bdist_wheel fo

In [0]:
import os
import numpy as np
import seaborn as sb
import pandas as pd
import matplotlib as mpl
import cvxpy
import cvxopt
import math
from cvxopt.modeling import variable
from numpy import float64, uint64
from cvxopt.modeling import op
from google.colab import files
import operator
import functools

In [11]:
ex1RawData = {'sportsMin': [2*10**6, 0, 1*10**6, 0], 'natMin': [1*10**6, 1*10**6, 1*10**6, 0], 'totMin' : [0*10**6, 2*10**6, 3*10**6, 2*10**6], 'sportsCThru' :[2.5*10**-2, 2*10**-2, 1*10**-2, 1.5*10**-2],
             'natCThru': [0.8*10**-2, 1*10**-2, 3*10**-2, 2*10**-2]}
print(ex1RawData)
ex1DF = pd.DataFrame(data=ex1RawData)

print(ex1DF)

{'sportsMin': [2000000, 0, 1000000, 0], 'natMin': [1000000, 1000000, 1000000, 0], 'totMin': [0, 2000000, 3000000, 2000000], 'sportsCThru': [0.025, 0.02, 0.01, 0.015], 'natCThru': [0.008, 0.01, 0.03, 0.02]}
   natCThru   natMin  sportsCThru  sportsMin   totMin
0     0.008  1000000        0.025    2000000        0
1     0.010  1000000        0.020          0  2000000
2     0.030  1000000        0.010    1000000  3000000
3     0.020        0        0.015          0  2000000


#question1

In [0]:

# Our objective: maximize profit
# Profit comes from clicks: Pr = 2.4*clicks

# Decision variables: how many displays we will give for each company in each of the two sections (Sports & National)
#   let n_ij be the number of displays for company i on section j. The i index is the order the companies are seen on the dataframe
#   the j is 0 for sports section, 1 for national section
#   so, n_00 is the displays for geico in the sports section (indeces start at 0).

# n_00 = ex1DF.at[0,'sportsMin']
# n_01 = ex1DF.at[0,'natMin']
# n_10 = ex1DF.at[1,'sportsMin']
# n_11 = ex1DF.at[1,'natMin']
# n_20 = ex1DF.at[2,'sportsMin']
# n_21 = ex1DF.at[2,'natMin']
# n_30 = ex1DF.at[3,'sportsMin']
# n_31 = ex1DF.at[3,'natMin']

n_00 = variable(1,'geicoS')
n_01 = variable(1,'geicoN')
n_10 = variable(1,'deltaS')
n_11 = variable(1,'deltaN')
n_20 = variable(1,'tMobS')
n_21 = variable(1,'tMobN')
n_30 = variable(1,'cOneS')
n_31 = variable(1,'cOneN')

  
#   The objective function now can be written as: Pr = 2.4*sum(n_ij*clickThru_ij) (summing for all i, j values)
  
#  Constraints: The minimum displays promised to each company (either by section or total) as seen on the Min sections of the dataframe
#   so:
c1 = n_00 >= 2*10**6 #sportsMin
c2 =  n_01 >= 1*10**6 #natMin


c3 = n_11 >= 1*10**6
c4 = n_10 + n_11 >= 2*10**6 #totMin

c5 = n_20 >= 1*10**6
c6 = n_21 >= 1*10**6
c7 = n_20 + n_21  >= 3*10**6


c8 = n_30 + n_31 >= 2*10**6
    
#   Also, our space for displays is finite, so that is another constraint.
#   Let's assume we run the problem for a maximum of 6 million displays on sports and another 8 on national news
#   This constraint can be written as:
#     sum(n_i0) <= 6*10**6 #summing for all values of i (company)
#     sum(n_i1) <= 8*10**6 #summing for all values of i (company)

c9 = n_00 + n_10 + n_20 + n_30 <= 6*10**6
c10 = n_01 + n_11 + n_21 + n_31 <= 8*10**6

#we cannot have negative impressions, so that's an addittional constraint
#I don't need to add that for all, since some already have a requirement of minimum that is > 0 
c11 = n_10 >= 0
c12 = n_30 >= 0
c13 = n_31 >= 0

In [13]:
#objective function, as described on cell above:


print(type(ex1DF.at[0,'sportsCThru'].item())) #need to convert to generic float, otherwise it doesn't work. That's what .item() does

ourProfitFormula =  2.4*(n_00*ex1DF.at[0,'sportsCThru'].item() + n_01*ex1DF.at[0,'natCThru'].item() + 
                         n_10*ex1DF.at[1,'sportsCThru'].item() + n_11*ex1DF.at[1,'natCThru'].item() +
                         n_20*ex1DF.at[2,'sportsCThru'].item() + n_21*ex1DF.at[2,'natCThru'].item() +
                         n_30*ex1DF.at[3,'sportsCThru'].item() + n_31*ex1DF.at[3,'natCThru'].item())


<class 'float'>


#question2

In [14]:
# >>> from cvxopt.modeling import op
# >>> x = variable()
# >>> y = variable()
# >>> c1 = ( 2*x+y <= 3 )
# >>> c2 = ( x+2*y <= 3 )
# >>> c3 = ( x >= 0 )
# >>> c4 = ( y >= 0 )
# >>> lp1 = op(-4*x-5*y, [c1,c2,c3,c4])
# >>> lp1.solve()
# >>> lp1.status



with open('output.txt','a+') as ourFile:
        ourFile.write('test')
        profitObject = op(-ourProfitFormula, [c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13])
        profitObject.solve()
        profitObject.status
        print(-profitObject.objective.value())
        ourFile.write(str((-profitObject.objective.value())) + '\n')


     pcost       dcost       gap    pres   dres   k/t
 0: -5.1531e+05 -1.5476e+06  1e+07  2e-02  3e+00  1e+00
 1: -5.2476e+05 -5.6887e+05  4e+05  1e-03  1e-01  4e+02
 2: -6.8706e+05 -6.9404e+05  1e+05  3e-04  3e-02  5e+03
 3: -7.2832e+05 -7.3031e+05  4e+04  9e-05  9e-03  2e+03
 4: -7.3840e+05 -7.3848e+05  2e+03  4e-06  4e-04  7e+01
 5: -7.3919e+05 -7.3919e+05  2e+01  4e-08  4e-06  8e-01
 6: -7.3920e+05 -7.3920e+05  2e-01  4e-10  4e-08  8e-03
Optimal solution found.
[ 7.39e+05]



In [0]:
#@title question3 ,To be revisited

#For sensitivity analysis, ideally we would need a for loop that runs through all the desired combinations.
#Is there any built in cvx package method for sensitivity analysis?
#On a notebook, that is not very comfortable, so I manually changed the scenarios on the existing code and wrote the resuluts in attached files 



In [15]:
with open('output.txt','a+') as ourFile:
    nList = [n_00, n_01, n_10, n_11, n_20, n_21, n_30, n_31]

    for n in nList:
      print(n.name, n.value)
      ourFile.write(str(n.name) +  os.linesep) #neither \n or linesep seem to work. Is it something with collab?
      ourFile.write(str(n.value)+  os.linesep)

geicoS [ 3.58e+06]

geicoN [ 1.00e+06]

deltaS [ 1.00e+06]

deltaN [ 1.00e+06]

tMobS [ 1.00e+06]

tMobN [ 4.42e+06]

cOneS [ 4.22e+05]

cOneN [ 1.58e+06]



In [0]:



files.download('output.txt') #doesn't work, need to revisit

#question4

The results correlate heavily with the click through rates, as expected, as clicks are directly proportional to revenue.

Specifically:

delta Sports is maxed out (based on constrains for max 2mill total)
T-mobile national, Geico Sports and Captical One National are our top assignments, as well as the top click through.

Some of the values are related to the caps I set by myself for total sports and national displays.

In [0]:
#@title Exercise2



In [0]:
g_SA = 4 #required hours for a Standard (S) product to be Grinded (g) at plant A
p_SA = 2 #p for polished
g_DA = 2
p_DA = 5
g_SB = 5
p_SB = 5
g_DB = 3
p_DB = 6

#question1

Objective 1: maximize profit for each unit separately, given the raw material assignments

Profit = 10*(StandardSalesA + StandardsSalesB) + 15*(DeluxeSalesA + DeluxeSalesB)

Constraints: Capacities, Raw material requirements, number of products produced must be an integer >=0

Decision variables: how many of each product will each plant produce


In [0]:
n_SA = cvxpy.Int() #how many standards products will A produce
n_DA = cvxpy.Int() 
n_SB = cvxpy.Int() 
n_DB = cvxpy.Int() 

c1 = n_SA*g_SA + n_DA*g_DA <= 80 #grinding hours constraint for plant A
c2 = n_SA*p_SA + n_DA*p_DA <= 60#polishing hours constraint for plant A
c3 = n_SB*g_SB + n_DB*g_DB <= 60 #grinding hours constraint for plant B 
c4 = n_SB*p_SB + n_DB*p_DB <= 75#polishing hours constraint for plant B

c5 = 4*(n_SA + n_DA) <= 75 #raw mat const. for A
c6 = 4*(n_SB + n_DB) <= 45 #raw mat const. for B

c7 = n_SA >=0 
c8 = n_DA >=0

c9 = n_SB >=0 
c10 = n_DB >=0 

ourObjectiveA = cvxpy.Maximize(10*n_SA + 15*n_DA )
ourObjectiveB = cvxpy.Maximize(10*n_SB + 15*n_DB )

consA = [c1,c2,c5,c7,c8]
consB = [c3,c4,c6,c9,c10]

#question2

In [18]:
#solving for each plant separately
ourProbA = cvxpy.Problem(ourObjectiveA, consA)
resultA = ourProbA.solve()
print(n_SA.value,'Std at A and', n_DA.value, 'Deluxe at A, with a profit of', ourObjectiveA.value) 

ourProbB = cvxpy.Problem(ourObjectiveB, consB)
resultB = ourProbB.solve()
print(n_SB.value,'Std at B and', n_DB.value, 'Deluxe at B with a profit of', ourObjectiveB.value) 

10.000000000492811 Std at A and 8.000000003148305 Deluxe at A, with a profit of 220.0000000521527
2.912859137415921e-09 Std at B and 11.000000002512746 Deluxe at B with a profit of 165.00000006681978


#question3

In [19]:
#Now, we want to solve globally. The difference is that we basically have control over the raw material assignment, so that is 2 additional decision variables
c11 = 4*(n_SA + n_DA) + 4*(n_SB + n_DB) <= 120
# rawA = variable(1) #not needed
# rawB = variable(1)
ourObjectiveGlobal = cvxpy.Maximize(10*(n_SA + n_SB) + 15*(n_DA + n_DB))
consGlobal = [c1,c2,c3,c4,c7,c8,c9,c10,c11]
ourProbGlobal = cvxpy.Problem(ourObjectiveGlobal,consGlobal)
resultGlobal = ourProbGlobal.solve()
print(n_SA.value,'std at A,', n_DA.value,'del at A,', n_SB.value,'std at B,', n_DB.value,'del at B,', ourObjectiveGlobal.value)
#from the result, we can also find the raw material disrtibution:
print( (10+8)*4, 'raw at A' )
print( 12*4, 'raw at B' )

9.999999999202235 std at A, 7.999999996534886 del at A, -1.4098916005133095e-09 std at B, 11.999999999969063 del at B, 399.99999992548265
72 raw at A
48 raw at B


#questions 4 and 5

As expected, the joint optimization provides more profits (400 vs 220+165 = 385). Obviously, joint optimization result is always >= individual, as long as we can reassign raw material at will.
It seems a small tweak of raw material assignments is needed.

Another thing to recommend to the CEO is doing something else with the space and resources used for producing standard products in Plant B, as based on the optimization, it is completely useless (0 units produced in the optimal solution)
Maybe convert it to produce Deluxe somehow, or sell the machines and free up space.

In case there is budget for addittional capacity to be created, we should check the market for elasticity of demand and start a new analysis to find by how much we should be increasing capacity.

In [0]:
#@title Exercise3


Profitability = earnings before interest and taxes divided by total revenue. Note that the profitability is %(max100), not a ratio(max1).

So this is a % of the revenue of the hotel, not absolute $

#**question 1**

**Variables that positively affect profitability: **

Price to buy the inn: Somewhat expected, but not totally. If profitability was $amount, that would be very expected, but for the ratio not so much, especially given how strong the correlation is.
It seems expensive inns outperform the smaller ones for the most part. I guess nicer hotels can operate on a higher profit margin.

College students in the area: This makes sense, because college students may have parents visiting, or universities may hold various scientific, cultural, sports and other events, for which people visit and spend the night at hotels.

**Variables that negatively affect profitability: **

Square root of the median income in the area: Makes sense, because people in such areas often are more price sensitive so the profit margins are thinner.

State population per inn: Somewhat makes sense. Larger state ---> More competition ---> Thinner profit margins

#**question 2**

Hotel	Location	Price	Price (normalized)	Square Root of Median Income (normalized)	College Students in Area (normalized)	State Population Per Inn (normalized)	**profitability**

1	Eureka, California	$2,925,000.00 	-0.30	-0.81	-0.54	-1.00	44.24236879

**2	Fresno, California	$10,000,000.00 	1.70	-0.41	0.31	-0.47	53.37919231 HIGHEST PROFITABILITY**

3	Fresno, California	$3,750,000.00 	-0.07	-0.41	0.31	-0.47	43.02117894

4	Fresno, California	$3,500,000.00 	-0.14	-0.41	0.31	-0.47	42.6068584

5	Fresno, California	$325,000.00 	-1.04	-0.41	0.31	-0.47	37.34498761

6	Long Beach, California	$8,950,000.00 	1.40	0.66	0.48	-0.56	49.09506947

7	Los Angeles, California	$1,950,000.00 	-0.58	0.17	3.11	3.11	23.77686566

**8	Los Angeles, California	$1,750,000.00 	-0.63	0.17	3.11	3.11	23.44540924 LOWEST PROFITABILITY**

9	Los Angeles, California	$4,900,000.00 	0.26	0.17	3.11	3.11	28.66584798

10	South Lake Tahoe, California	$1,650,000.00 	-0.66	-0.79	-0.59	-0.43	38.88067311

11	South Lake Tahoe, California	$1,125,000.00 	-0.81	-0.79	-0.59	-0.43	38.01059999

12	South Lake Tahoe, California	$2,500,000.00 	-0.42	-0.79	-0.59	-0.43	40.28936293

13	South Lake Tahoe, California	$1,975,000.00 	-0.57	-0.79	-0.59	-0.43	39.41928981

14	South Lake Tahoe, California	$3,750,000.00 	-0.07	-0.79	-0.59	-0.43	42.36096561

15	South Lake Tahoe, California	$1,475,000.00 	-0.71	-0.79	-0.59	-0.43	38.59064874

16	South Lake Tahoe, California	$750,000.00 	-0.92	-0.79	-0.59	-0.43	37.38911919




#**question3**

a. With USD10M budget, following the strategy described, we would buy the large hotel (2) that costs exactly 10M and nothing else.

Total profitability 53.8% of revenue

b. A bit confused how total profitability is defined as a sum here. Based on the definition of profitability as a ratio (% - quoting: *“Profitability”, computed at the operating
margin, or earnings before interest and taxes **divided** by total revenue.), the total profitability should be the weighted (on revenue) average of profitability of different hotels.*

c. If the profitability is indeed the ratio (%) I assumed, and we are confident about the model, and we don't care about diversification in locations, then this is a good approach. If we had 20M instead of 10M, we would just continue buying based on the highest profitability %

If we were to sum the profitabilites, then it seems like a bad approach, as there are multiple combinations that would provide a higher sum.

#**question4**

Based on the description, we want to optimize the **sum** of profitabilities. One way to do that is to find the following ratio:

(profitability/price)*10^6   Then we can brute force from the highest to the lowest and start buying. Only for the last purchase, we need to make sure it doesn't leave us with leftover cash (if we do not want that). So only for the last choice, we may choose a hotel with a ratio that is not the highest. However, that is an analytical solution that does not require linear programming.

15.12559617

5.337919231

11.47231438

12.17338811

114.9076542

5.485482622

12.19326444

13.39737671

5.850173056

23.56404431

33.78719999

16.11574517

19.95913408

11.29625749

26.16315169

49.85215892

For the linear programming way:

The **objective function** (to be maximized) will be:

Total Profitability = Sum(Profitability_j) where j the index of selected hotels.


**Decision Variables:**

b_i = 1 if you buy hotel i, or = 0 if you don't buy it.

So the objective function becomes: Total Profitability =  sum(b_i*Profitability_i) , with i any hotel index (as the ones we do not buy have b_i=0 anyway)

**Constraints:**

sum(Price_j) <= 10*10^6

b_i = 0 or 1 (binary variable)

In [29]:
iHotRange = range(1,17) #range 1,17 gives us 1 to 16 in Python
list_b = list()

for iHotel in iHotRange:
  b_i = cvxpy.Bool()
  list_b.append(b_i)
  #b_i = 1 #test to see how Bool behaves when assigned 1 or 0. No error
  
print(list_b)



[Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1)]


In [23]:
pr1 = 44.24236879 #values from LMQ.xlsx (calculated by me based on the definition)
pr2 = 53.37919231
pr3 = 43.02117894
pr4 = 42.6068584
pr5 = 37.34498761
pr6 = 49.09506947
pr7 = 23.77686566
pr8 = 23.44540924
pr9 = 28.66584798
pr10 = 38.88067311
pr11 = 38.01059999
pr12 = 40.28936293
pr13 = 39.41928981
pr14 = 42.36096561
pr15 = 38.59064874
pr16 = 37.38911919

list_prof = [pr1, pr2, pr3, pr4, pr5, pr6, pr7, pr8, pr9, pr10, pr11, pr12, pr13, pr14, pr15, pr16]
print(list_prof)


import operator
import functools



[44.24236879, 53.37919231, 43.02117894, 42.6068584, 37.34498761, 49.09506947, 23.77686566, 23.44540924, 28.66584798, 38.88067311, 38.01059999, 40.28936293, 39.41928981, 42.36096561, 38.59064874, 37.38911919]


In [0]:
def sumproduct(*lists): #generalized way to do sumproduct of lists without manually writing down each element
    return sum(functools.reduce(operator.mul, data) for data in zip(*lists))
  
ourProfitSum = sumproduct(list_b, list_prof)
ourObjectiveHotels = cvxpy.Maximize(ourProfitSum)

list_prices = [2925000.00,
10000000.00,
3750000.00,
3500000.00,
325000.00,
8950000.00,
1950000.00,
1750000.00,
4900000.00,
1650000.00,
1125000.00,
2500000.00,
1975000.00,
3750000.00,
1475000.00,
750000.00]

cH1 = sumproduct(list_b, list_prices) <= 10*10**6
consHotels = list()
consHotels.append(cH1)

# ourObjectiveGlobal = cvxpy.Maximize(10*(n_SA + n_SB) + 15*(n_DA + n_DB)) #for reference, ignore
# consGlobal = [c1,c2,c3,c4,c7,c8,c9,c10,c11]
# ourProbGlobal = cvxpy.Problem(ourObjectiveGlobal,consGlobal)
# resultGlobal = ourProbGlobal.solve()

In [60]:
ourProbHotels = cvxpy.Problem(ourObjectiveHotels, consHotels)
resultHotels =  ourProbHotels.solve()

# print(n_SA.value,'std at A,', n_DA.value,'del at A,', n_SB.value,'std at B,', n_DB.value,'del at B,', ourObjectiveGlobal.value) #ignore
# #from the result, we can also find the raw material disrtibution:
# print( (10+8)*4, 'raw at A' )
# print( 12*4, 'raw at B' )
list_b_converted = list()
for i1 in range(0,16):
  print(i1, list_b[i1].value) #remember, 0 (or close to it), we don't buy it. 1 (or close to it) we buy it.
  list_b_converted.append(list_b[i1].value) #taking the value from vars, need this for the next calculation
  
ourResultProfitabilitySum1 = sumproduct(list_b_converted, list_prof)
print('the sum of profitability of selected hotels is', ourResultProfitabilitySum1)

0 1.0931077598554305e-13
1 -8.041165148868167e-14
2 2.2123295858170557e-13
3 2.0735024101201292e-13
4 0.9999999999910595
5 -3.704815689124423e-14
6 2.138413386106903e-13
7 1.626034121418402e-13
8 1.927892693700597e-13
9 0.9999999999996143
10 0.9999999999988642
11 1.0000000000000584
12 0.9999999999998463
13 2.2458273861807826e-13
14 0.9999999999994341
15 0.9999999999975361
the sum of profitability of selected hotels is 269.92468137953136


If we accept the sum of profitabilites as what we need, then the solution makes sense intuitively. This basically tells us to buy as many small hotels as we can (and sum up a large profitability score).
The total score of almost 270 >> 53 of the original solution. However, with weighted average this would be very different.

#question 5

We need a location id, let's call it l:

Eureka = 0

Fresno = 1

Long Beach = 2

LA = 3

South Lake Tahoe = 4


The new constraint is:

sum(b_i_l) <=2 for any specific, given value of location id l

The new solution has to be equal or worse to the previous one, as we added another constraint without altering the problem otherwise.

In [0]:
list_l = [0,1,1,1,1,2,3,3,3,4,4,4,4,4,4,4] 

cH_loc1 = list_b_converted[1] + list_b_converted[2] + list_b_converted[3] + list_b_converted[4] <= 2.1 #faster to do it manually in this case. location1 (0skipped cause its less than 2 options anyway)
cH_loc3 = list_b_converted[6] + list_b_converted[7] + list_b_converted[8]<= 2.1 # location 3 (2 skipped)
cH_loc4 = list_b_converted[15] + list_b_converted[14] + list_b_converted[13] + list_b_converted[12] + list_b_converted[11] + list_b_converted[10] + list_b_converted[9]<= 2.1 #location 4
#2.1 instead of 2 to be on the safe side with approx

consHotels2 = consHotels


In [0]:
consHotels2.append(cH_loc1)
consHotels2.append(cH_loc3)
consHotels2.append(cH_loc4)

In [70]:


iHotRange = range(1,17) #range 1,17 gives us 1 to 16 in Python
list_b2 = list()
list_b2_int = list()
for iHotel in iHotRange:
  b_i2 = cvxpy.Bool()
  list_b2.append(b_i2)
  #b_i = 1 #test to see how Bool behaves when assigned 1 or 0. No error
  
print(list_b2) #Bool is creating problems with calculations. AttributeError: 'numpy.bool_' object has no attribute 'variables'. 
#We could model it as Int, then add constraints >=0 and <=1, but this not a clean way to handle. Better find a conversions Bool --> Int (CVXPY types)
print(list_b2[1].value) #testing

ourProfitSum2 = sumproduct(list_b2, list_prof)
ourObjectiveHotels2 = cvxpy.Maximize(ourProfitSum2)
list_b_converted2 = list()

ourProbHotels2 = cvxpy.Problem(ourObjectiveHotels2, consHotels2) #new constraints, everything else the same
resultHotels2 =  ourProbHotels2.solve()


for i2 in range(0,16):
  print(i2, list_b2[i2].value) #remember, 0 (or close to it), we don't buy it. 1 (or close to it) we buy it.
  list_b_converted.append(list_b[i2].value) #taking the value from vars, need this for the next calculation
  
ourResultProfitabilitySum2 = sumproduct(list_b_converted2, list_prof)
print('the sum of profitability of selected hotels is', ourResultProfitabilitySum2)

[Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1), Bool(1, 1)]
None


AttributeError: ignored

#question 6


If we are confident about the sum of profitability, I would say that it makes sense to pick the many small hotels solution. It also offers diversification in location.
I have not seen any EDA on how this model was created, so any suggestions would be tenative. We would need to see more historical data to see how this model was derived.
It seems that some variables are very strongly represented (>5 factor) and it would be good to reexamine how reliable that is based on the data. 


Also, when using linear models, we need to make sure the relationships are actually linear (or close enough to it). If for some of the variables it is not, we could use a sqrt or a log transformation, or switch to a no linear model.


As for the optimization model, we could take more factors into account, such as volatility in profitability over time, areas that are more up and coming, a minumum or maximum spend on an individual hotel.